# Beam center finder

In SANS experiments, it is essential to find the center of the scattering pattern in order to allow symmetric summation of the scattering intensity around the beam (i.e. computing a one-dimensional $I(Q)$).
As detector panels can move, the center of the beam will not always be located at the same place on the detector panel from one experiment to the next.

Here we describe two different algorithms that can be used to determine the position of the beam center:

1. using the center of mass of the pixel counts
1. using an iterative refinement on a computed scattering cross-section to find the center of the scattering pattern

In [ ]:
import scipp as sc
import plopp as pp
import sciline as sl
import esssans as sans
from esssans.isis import plot_flat_detector_xy
from esssans.types import *

We begin by setting some parameters relevant to the current sample.

In [ ]:
params = sans.sans2d.default_parameters.copy()

params[FileList[SampleRun]] = ['SANS2D00063114.hdf5']
params[sans.sans2d.LowCountThreshold] = sc.scalar(100.0, unit='counts')

In [ ]:
# # Include effects of gravity?
# gravity = True

# # Wavelength binning
# wavelength_bins = sc.linspace(
#     dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom'
# )

# # Define Q binning
# q_bins = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

# # Define coordinate transformation graph
# graph = sans.conversions.sans_elastic(gravity=gravity)

Next we load the data files for the sample and direct runs:

In [ ]:
# sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.hdf5'))
# direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.hdf5'))
# # Add X, Y coordinates
# sample.coords['x'] = sample.coords['position'].fields.x
# sample.coords['y'] = sample.coords['position'].fields.y
# # Add gravity coordinate
# sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g

### Masking bad pixels

We create a quick image of the data (summing along the `tof` dimension) to inspect its contents.
We see a diffuse scattering pattern, centered around a dark patch with an arm to the north-east; this is the sample holder.
It is clear that the sample and the beam are not in the center of the panel, which is marked by the red dot

In [ ]:
providers = list(sans.providers + sans.sans2d.providers)
pipeline = sciline.Pipeline(providers, params=params)

In [ ]:
pipeline.visualize(MaskedData[SampleRun], graph_attr={'rankdir': 'LR'})

In [ ]:
raw = pipeline.compute(RawData[SampleRun])

p = plot_flat_detector_xy(raw.sum('tof'), norm='log')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

To avoid skew in future comparisons of integrated intensities between the different regions of the detector panel,
we mask out the sample holder, using a low-counts threshold.
This also masks out the edges of the panel, which show visible artifacts.
We also mask out a region in the bottom right corner where a group of hot pixels is apparent.
Finally, there is a single hot pixel in the detector on the right edge of the panel with counts in excess of 1000,
which we also remove.

In [ ]:
masked = pipeline.compute(MaskedData[SampleRun])

p = plot_flat_detector_xy(masked.sum('tof'), norm='log')
p.ax.plot(0, 0, 'o', color='violet', ms=5)
p

In [ ]:
masked.masks['low_counts'] = masked.sum('tof').data < sc.scalar(70., unit='counts')

p = plot_flat_detector_xy(masked.sum('tof'), norm='log')
p.ax.plot(0, 0, 'o', color='violet', ms=5)
p

## First method: center-of-mass calculation

In [ ]:
pipeline.visualize(BeamCenter, graph_attr={'rankdir': 'LR'})

In [ ]:
com = pipeline.compute(BeamCenter)
com

In [ ]:
xc = com.fields.x
yc = com.fields.y
p.ax.plot(xc.value, yc.value, 'o', color='r', ms=5)
p

## Second method: refinement using I(Q) inside 4 quadrants

### Description of the procedure

The procedure to determine the precise location of the beam center is the following:

1. use the center-of-mass as an initial guess
2. from that initial guess, divide the panel into 4 quadrants
3. compute $I(Q)$ inside each quadrant and compute the residual difference between all 4 quadrants
4. iteratively move the centre position and repeat 2. and 3. until all 4 $I(Q)$ curves lie on top of each other

In [ ]:
params[WavelengthBins] = sc.linspace(
    'wavelength', start=2.0, stop=16.0, num=141, unit='angstrom'
)
params[WavelengthMask] = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[2.21, 2.59], unit='angstrom'
        )
    },
)
params[FileList[TransmissionRun[SampleRun]]] = params[FileList[SampleRun]]
params[FileList[EmptyBeamRun]] = ['SANS2D00063091.hdf5']

params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

params[sans.beam_center_finder.BeamCenterFinderQBins] = sc.linspace(
        'Q', 0.02, 0.3, 71, unit='1/angstrom'
    )

providers.remove(sans.beam_center_finder.beam_center_from_center_of_mass)
providers.append(sans.beam_center_finder.beam_center_from_iofq)
pipeline = sciline.Pipeline(providers, params=params)
pipeline.visualize(BeamCenter, graph_attr={'rankdir': 'LR'})

### Making 4 quadrants

We divide the panel into 4 quadrants.

In [ ]:
p = plot_flat_detector_xy(masked.sum('tof'), norm='log')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p.ax.axvline(xc.value, color='cyan')
p.ax.axhline(yc.value, color='cyan')
dx = 0.25
style = dict(ha='center', va='center', color='w')
p.ax.text(xc.value + dx, yc.value + dx, 'North-East', **style)
p.ax.text(xc.value - dx, yc.value + dx, 'North-West', **style)
p.ax.text(xc.value + dx, yc.value - dx, 'South-East', **style)
p.ax.text(xc.value - dx, yc.value - dx, 'South-West', **style)
p

### Compute $I(Q)$ inside the 4 quadrants

We begin by defining several parameters which are required to compute $I(Q)$.

In [ ]:
kwargs = dict(data= masked,
norm=pipeline.compute(NormWavelengthTerm[SampleRun]),
graph=pipeline.compute(sans.conversions.ElasticCoordTransformGraph),
q_bins=params[sans.beam_center_finder.BeamCenterFinderQBins],
wavelength_bins=params[WavelengthBins],
transform=pipeline.compute(LabFrameTransform[SampleRun]),
pixel_shape=pipeline.compute(DetectorPixelShape[SampleRun]))

In [ ]:
from esssans.beam_center_finder import _iofq_in_quadrants

results = _iofq_in_quadrants(
    xy=[xc.value, yc.value],
    **kwargs,
)

We can now plot on the same figure all 4 $I(Q)$ curves for each quadrant.

In [ ]:
pp.plot(results, norm='log')

As we can see, the overlap between the curves from the 4 quadrants is not satisfactory.
We will now use an iterative procedure to improve our initial guess, until a good overlap between the curves is found.

For this, we first define a cost function, which gives us an idea of how good the overlap is:

$$
\text{cost} = \frac{\sum_{Q}\sum_{i=1}^{i=4} \overline{I}(Q)\left(I(Q)_{i} - \overline{I}(Q)\right)^2}{\sum_{Q}\overline{I}(Q)} ~,
$$

where $\overline{I}(Q)$ is the mean intensity of the 4 quadrants (represented by $i$) as a function of $Q$.
This is basically a weighted mean of the square of the differences between the $I(Q)$ curves in the 4 quadrants with respect to $\overline{I}(Q)$,
and where the weights are $\overline{I}(Q)$.

Next, we iteratively minimize the computed cost
(this is using Scipy's `optimize.minimize` utility internally;
see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) for more details).

In [ ]:
xc

In [ ]:
from scipy.optimize import minimize

# The minimizer works best if given bounds, which are the bounds of our detector panel
x = masked.coords['position'].fields.x
y = masked.coords['position'].fields.y

res = minimize(
    sans.beam_center_finder._cost,
    x0=[xc.value, yc.value],
    args=tuple(kwargs.values()),
    bounds=[(x.min().value, x.max().value), (y.min().value, y.max().value)],
    method='Nelder-Mead',
    tol=0.01,
)

res

Once the iterations completed, the returned object contains the best estimate for the beam center:

In [ ]:
res.x

We can now feed this value again into our `iofq_in_quadrants` function, to inspect the $Q$ intensity in all 4 quadrants:

In [ ]:
xy = [ 0.09329701, -0.08079138]

results = _iofq_in_quadrants(
    # xy=[res.x[0], res.x[1]],
    xy=xy,
    **kwargs,
)

pp.plot(results, norm='log')

The overlap between the curves is excellent, allowing us to safely perform an azimuthal summation of the counts around the beam center.

As a consistency check, we plot the refined beam center position onto the detector panel image:

In [ ]:
p = sample.sum('tof').copy().hist(y=120, x=128).plot(norm='log', aspect='equal')
p.ax.plot(res.x[0], res.x[1], 'o', color='red', ms=5)
p

In [ ]:
p = plot_flat_detector_xy(masked.sum('tof'), norm='log')
p.ax.plot(res.x[0], res.x[1], 'o', color='red', ms=5)
# p.ax.axvline(xc.value, color='cyan')
# p.ax.axhline(yc.value, color='cyan')
p

## Footnotes

<div id='footnote1'></div>

1. In the full $I(Q)$ reduction, there is a term $D(\lambda)$ in the normalization called the "direct beam" which gives the efficiency of the detectors as a function of wavelength.
Because finding the beam center is required to compute the direct beam in the first place,
we do not include this term in the computation of $I(Q)$ for finding the beam center.
This changes the shape of the $I(Q)$ curve, but since it changes it in the same manner for all $\phi$ angles,
this does not affect the results for finding the beam center.